In [ ]:
import numpy as np
import scipy.signal
import socket
import numpy as np


In [ ]:


# Step 1: Simulate capturing analog voice input
def capture_voice_input():
    # This function would typically capture analog voice input from a microphone or another source
    # For simulation purposes, we'll generate random analog voice data
    analog_voice_data = np.random.randint(-32768, 32767, size=(44100,))
    return analog_voice_data

# Step 2: Digitize analog voice data
def digitize_voice_data(analog_voice_data):
    # This function digitizes the analog voice data using an ADC
    # For simulation purposes, we'll simply quantize the analog data to 16 bits
    digital_voice_data = np.round(analog_voice_data / 32768 * 32767)
    return digital_voice_data.astype(int)

# Step 3: Preprocess digital voice data
def preprocess_voice_data(digital_voice_data):
    # Apply some audio processing techniques, such as filtering or normalization
    # For simulation purposes, we'll apply a simple normalization to scale the voice data to [-1, 1]
    voice_data_normalized = digital_voice_data / np.max(np.abs(digital_voice_data))
    return voice_data_normalized

# Step 4: Simulate modulation onto laser beam (with additional parameters)
def modulate_signal(caller_phone_number, receiver_phone_number, voice_data):
    # Ensure voice data is converted to a string (assuming voice_data is numeric)
    voice_data_str = str(voice_data)

    # Concatenate caller's phone number, receiver's phone number, and voice data
    signal_data = f"{caller_phone_number}-{receiver_phone_number}-{voice_data_str}"

    # Return the modulated signal
    return signal_data

# Example usage:
def main():
    # Server address and port
    server_address = ('127.0.0.1', 5556)

    # Create a socket object
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    try:
        # Connect to the server
        client_socket.connect(server_address)
        print("Connected to server.")

        # Simulate initiating a call
        caller_phone_number = input('Enter your phone number: ')
        receiver_phone_number = input('Enter the receiver\'s phone number: ')

        # Simulate capturing analog voice input
        analog_voice_data = capture_voice_input()

        # Digitize analog voice data
        digital_voice_data = digitize_voice_data(analog_voice_data)

        # Preprocess digital voice data
        processed_voice_data = preprocess_voice_data(digital_voice_data)

        # Simulate modulation onto laser beam with caller and receiver phone numbers
        modulated_signal = modulate_signal(caller_phone_number, receiver_phone_number, processed_voice_data)

        # Send the modulated signal to the server
        client_socket.sendall(modulated_signal.encode('utf-8'))
        print("Modulated signal sent to server.")

    finally:
        # Close the socket
        client_socket.close()

if __name__ == '__main__':
    main()


In [1]:
import socket
import threading
import tkinter as tk
import pyaudio

# Server configuration
SERVER_HOST = '127.0.0.1'  # Server IP
SERVER_PORT = 12345

# PyAudio configuration
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 1024

# Create a PyAudio object
audio = pyaudio.PyAudio()

# Global variables
stream = None

# Function to handle receiving audio from the server
# Function to handle receiving audio from the server
def receive_audio():
    global stream
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, output=True, frames_per_buffer=CHUNK)
    
    while True:
        try:
            data = client_socket.recv(CHUNK)
            if not data:
                break
            stream.write(data)
        except Exception as e:
            print("Error receiving audio:", e)
            break


# Function to start recording audio
def start_recording():
    global stream
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
    threading.Thread(target=send_audio).start()

# Function to stop recording audio
def stop_recording():
    global stream
    if stream is not None:
        stream.stop_stream()
        stream.close()

# Function to send recorded audio to the server
def send_audio():
    global stream
    while True:
        try:
            data = stream.read(CHUNK)
            client_socket.sendall(data)
        except Exception as e:
            print("Error sending audio:", e)
            break

# Function to create GUI
def create_gui():
    # Create main window
    root = tk.Tk()
    root.title("Voice Chat")

    # Create buttons
    start_button = tk.Button(root, text="Start Recording", command=start_recording)
    start_button.pack()

    stop_button = tk.Button(root, text="Stop Recording", command=stop_recording)
    stop_button.pack()

    # Run the GUI
    root.mainloop()

# Connect to the server
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((SERVER_HOST, SERVER_PORT))

# Start a thread to receive audio from the server
threading.Thread(target=receive_audio).start()

# Create GUI
create_gui()

# Cleanup
audio.terminate()
client_socket.close()


Error receiving audio: [WinError 10054] An existing connection was forcibly closed by the remote host
Error sending audio: [WinError 10054] An existing connection was forcibly closed by the remote host
